<a href="https://colab.research.google.com/github/xren451/DAMR/blob/main/DAMR_AIRQUALITY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#DAMR code

In [2]:
!git clone https://github.com/xren451/DAMR.git

fatal: destination path 'DAMR' already exists and is not an empty directory.


In [3]:
from google.colab import drive
import sys
drive.mount('/content/drive')

if not 'DAMR' in sys.path:
    sys.path += ['DAMR']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#DAMR

Hi, here is the code of DAMR (Dynamic Ajacency Matrix Representation) for data imputation on imputation task.


In [4]:
!pip install spektral
!pip install fancyimpute

# 1. Import library

In [5]:
#1.Import library
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
import copy
from sklearn.preprocessing import StandardScaler
import scipy.special as special
import scipy.sparse as sp
import pickle
import os
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from scipy.fftpack import fft
import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, GRU, Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, LeakyReLU, GRU, Concatenate, Reshape, Softmax, Attention
from tensorflow.keras import activations
from tensorflow.keras.models import Model
#from keras.layers import LeakyReLU
import spektral
from spektral.layers import  GCSConv, DiffusionConv, GATConv, ARMAConv, GCNConv
from tensorflow.keras.regularizers import l2
#from my own library import Generate_missing and other models
from model import * #generate_miss(data,perc),output:data after generate missing.
import FancyImpute
from FancyImpute import *
import torch
#Bidirectional LSTM
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
import pickle

In [15]:
# 2. Data Pre-processing.

In [17]:
#Define hyperparameters by widgets in jupyter notebook
import ipywidgets as widgets
from IPython.display import display

# Create widgets for each parameter
Miss_perc_widget = widgets.FloatSlider(value=0.1,description='Miss_perc:')
Split_perc1_widget = widgets.FloatSlider(value=0.8,  description='Split_perc1:')
Split_perc2_widget = widgets.FloatSlider(value=0.1,  description='Split_perc2:')
learning_rate_widget = widgets.FloatSlider(value=0.0001,  description='Learning Rate:')
batch_size_widget = widgets.IntSlider(value=24,description='Batch Size:')
epochs_widget = widgets.IntSlider(value=100,description='Epochs:')
seed_widget = widgets.IntSlider(value=42, description='Seed:')
verbose_widget = widgets.IntSlider(value=1,description='Verbose:')
data_path_widget = widgets.Text(value='/content/DAMR/Data/Airquality/raw', description='Data Path:')

# Display widgets
display(Miss_perc_widget, Split_perc1_widget, Split_perc2_widget, learning_rate_widget, batch_size_widget,
        epochs_widget, seed_widget, verbose_widget)

# Access values using widget.value
Miss_perc = Miss_perc_widget.value
Split_perc1 = Split_perc1_widget.value
Split_perc2 = Split_perc2_widget.value
learning_rate = learning_rate_widget.value
batch_size = batch_size_widget.value
epochs = epochs_widget.value
seed = seed_widget.value
verbose = verbose_widget.value
path = data_path_widget.value

FloatSlider(value=0.1, description='Miss_perc:')

FloatSlider(value=0.8, description='Split_perc1:')

FloatSlider(value=0.1, description='Split_perc2:')

FloatSlider(value=0.0001, description='Learning Rate:')

IntSlider(value=24, description='Batch Size:')

IntSlider(value=100, description='Epochs:')

IntSlider(value=42, description='Seed:')

IntSlider(value=1, description='Verbose:')

In [14]:
data=Csv2Tensor(path)
data=data[:,:,:]
print(data.shape)
#Generate Missing 3D.
COVID_features=data
COVID_features=MinMax3D(COVID_features)#(503,3,7)
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)

()
()
()
()
()
()
()
(503, 3, 7)
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


In [20]:
a=[[0,"inf",1],[1,"inf",0]]
a=np.array(a)
print(np.array(a).shape[0])
for i in range(np.array(a).shape[0]):
    for j in range(np.array(a).shape[1]):
        if a[i,j]=="inf":
            a[i,j]=np.nan
print(a)

2
[['0' 'nan' '1']
 ['1' 'nan' '0']]


In [27]:
#Run baseline of MEAN
MEAN_RMSE,MEAN_MAE,MEAN_MAPE,MEAN_MRE=MEAN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
print("MEAN_RMSE",MEAN_RMSE,"MEAN_MAE",MEAN_MAE,"MEAN_MAPE",MEAN_MAPE,"MEAN_MRE",MEAN_MRE)

#Run baseline(KNN,MICE,MF,MEAN,SLIDING)#Return:RMSE,MAE,MAPE,MRE
#For the model of FancyImpute,we just need to test.

KNN_RMSE,KNN_MAE,KNN_MAPE,KNN_MRE=KNN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MICE_RMSE,MICE_MAE,MICE_MAPE,MICE_MRE=MICE_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
#MF_RMSE,MF_MAE,MF_MAPE,MF_MRE=MF_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MEAN_RMSE,MEAN_MAE,MEAN_MAPE,MEAN_MRE=MEAN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask,40)
print("KNN_RMSE",KNN_RMSE,"KNN_MAE",KNN_MAE,"KNN_MAPE",KNN_MAPE,"KNN_MRE",KNN_MRE)
print("MICE_RMSE",MICE_RMSE,"MICE_MAE",MICE_MAE,"MICE_MAPE",MICE_MAPE,"MICE_MRE",MICE_MRE)
#print("MF_RMSE",MF_RMSE,"MF_MAE",MF_MAE,"MF_MAPE",MF_MAPE,"MF_MRE",MF_MRE)
print("MEAN_RMSE",MEAN_RMSE,"MEAN_MAE",MEAN_MAE,"MEAN_MAPE",MEAN_MAPE,"MEAN_MRE",MEAN_MRE)
print("Sliding_RMSE",Sliding_RMSE,"Sliding_MAE",Sliding_MAE,"Sliding_MAPE",Sliding_MAPE,"Sliding_MRE",Sliding_MRE)

MEAN_RMSE 0.20723746404543353 MEAN_MAE 0.1418540127268051 MEAN_MAPE 0.3841831646744543 MEAN_MRE 0.21125705156044017
Imputing row 1/731 with 0 missing, elapsed time: 0.289
Imputing row 101/731 with 1 missing, elapsed time: 0.307
Imputing row 201/731 with 0 missing, elapsed time: 0.309
Imputing row 301/731 with 0 missing, elapsed time: 0.326
Imputing row 401/731 with 1 missing, elapsed time: 0.328
Imputing row 501/731 with 1 missing, elapsed time: 0.333
Imputing row 601/731 with 0 missing, elapsed time: 0.342
Imputing row 701/731 with 0 missing, elapsed time: 0.344
Imputing row 1/731 with 1 missing, elapsed time: 0.254
Imputing row 101/731 with 1 missing, elapsed time: 0.265
Imputing row 201/731 with 1 missing, elapsed time: 0.269
Imputing row 301/731 with 1 missing, elapsed time: 0.276
Imputing row 401/731 with 0 missing, elapsed time: 0.280
Imputing row 501/731 with 1 missing, elapsed time: 0.282
Imputing row 601/731 with 0 missing, elapsed time: 0.285
Imputing row 701/731 with 2 missi

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warning

KNN_RMSE 0.14011655638035272 KNN_MAE 0.08657806714759345 KNN_MAPE 0.21532546036872552 KNN_MRE 0.12893697431476486
MICE_RMSE 0.13172267745435567 MICE_MAE 0.08829627053233502 MICE_MAPE 0.21508108467128295 MICE_MRE 0.13149582037110266
MEAN_RMSE 0.20723746404543353 MEAN_MAE 0.1418540127268051 MEAN_MAPE 0.3841831646744543 MEAN_MRE 0.21125705156044017
Sliding_RMSE 0.24878518408284855 Sliding_MAE 0.17299590009614468 Sliding_MAPE inf Sliding_MRE 0.2999111383736634


In [28]:
#Create lags for the first variable.
#Extract the first variable
lags=30
var=0
Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_miss,COVID_features,COVID_features_mask,6)
Sliding_out_var1=Sliding_out[:,var,:]
Sliding_out_var1_sup=series_to_supervised(Sliding_out_var1,lags,1)
Sliding_out_var1_sup=Sliding_out_var1_sup.iloc[:,:lags*Sliding_out.shape[2]]
print(Sliding_out_var1_sup.shape)
#Input1: 473*7*30
COVID_X1_all_aftslid=np.array(Sliding_out_var1_sup).reshape(len(Sliding_out_var1_sup),COVID_features.shape[2],lags)
#Input2: 473*3*7
COVID_features_aftslid=Sliding_out[30:,:,:]
#Input3: Adj: 473*7*7(Initialization)
Adj_all = np.arange(len(COVID_features_aftslid)*COVID_features_aftslid.shape[2]*COVID_features_aftslid.shape[2]).reshape(len(COVID_features_aftslid),COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])#(473,7,7)
#Output:Groundtruth of the first variable->
COVID_Y1_all=COVID_features[30:,:,:]
COVID_Y1_all=COVID_Y1_all[:,var,:]
COVID_Y1_all=COVID_Y1_all.reshape(COVID_Y1_all.shape[0],COVID_Y1_all.shape[1],1)
#
print(COVID_X1_all_aftslid.shape,COVID_features_aftslid.shape,Adj_all.shape,COVID_Y1_all.shape)

/content/DAMR/FancyImpute.py:170: RuntimeWarning: divide by zero encountered in divide
  Sliding_mape= abs((np.array(Data_true)[missing_mask]-np.array(Data_Sliding)[missing_mask])/np.array(Data_true)[missing_mask]).mean()
/content/DAMR/FancyImpute.py:170: RuntimeWarning: divide by zero encountered in divide
  Sliding_mape= abs((np.array(Data_true)[missing_mask]-np.array(Data_Sliding)[missing_mask])/np.array(Data_true)[missing_mask]).mean()
/content/DAMR/FancyImpute.py:170: RuntimeWarning: divide by zero encountered in divide
  Sliding_mape= abs((np.array(Data_true)[missing_mask]-np.array(Data_Sliding)[missing_mask])/np.array(Data_true)[missing_mask]).mean()
/content/DAMR/FancyImpute.py:170: RuntimeWarning: divide by zero encountered in divide
  Sliding_mape= abs((np.array(Data_true)[missing_mask]-np.array(Data_Sliding)[missing_mask])/np.array(Data_true)[missing_mask]).mean()
/content/DAMR/FancyImpute.py:170: RuntimeWarning: divide by zero encountered in divide
  Sliding_mape= abs((np.a

(7266, 300)
(7266, 10, 30) (7266, 6, 10) (7266, 10, 10) (7266, 10, 1)


/content/DAMR/FancyImpute.py:170: RuntimeWarning: divide by zero encountered in divide
  Sliding_mape= abs((np.array(Data_true)[missing_mask]-np.array(Data_Sliding)[missing_mask])/np.array(Data_true)[missing_mask]).mean()


#3. Dynamic Adjacency matrix layer

In [29]:
#3.1 Distance-based adjacency matrix
#Lat and long of the specific dataset.
COVID_lat=pd.DataFrame([40.00395,39.914409,39.982053,39.954047,
39.878193,39.855958,39.939554,39.794491,39.885241,39.886491])
COVID_long=pd.DataFrame([116.20531,116.184239,116.3974,116.348991
,116.351974,116.36781,116.483746,116.506319,116.664162,116.407355])
#Adj_dist=adj_dist(COVID_lat,COVID_long)#10*10
#3.1 Distance-based adjacency matrix
#places:7 stations all over the world.
#COVID_lat=pd.DataFrame([-14.235,-35.6751,20.5937,32.4279,23.6345,-9.19,30.5595])
#COVID_long=pd.DataFrame([-51.9253,-71.543,78.9629,53.688,-102.5528,-75.0152,22.9375])
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7
Adj_dist.shape

(10, 10)

In [31]:
#3.2 Mean MI-based adjacency matrix
print(COVID_features_aftslid.shape)
MeanMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        MeanMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
print(np.array(MeanMI).shape)
MeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
print(MeanMI.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(MeanMI)
MeanMI=scaled1#shape:7*7

(7266, 6, 10)
(100,)
(10, 10)


In [32]:
#3.3 CONCAT MI-based adjacency matrix
print(COVID_features_aftslid.shape)
ConcatMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        ConcatMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
ConcatMI=np.array(ConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(ConcatMI)
ConcatMI=scaled1#shape:7*7

(7266, 6, 10)


In [33]:
#3.4 Single Feature MI-based adjacency matrix
COVID_features_aftslid.shape
FeatureMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        FeatureMI.append(Global_EachMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
#For the first feature,if choose the second feature, modify here!!
FeatureMI=np.array(FeatureMI)[:,2]
FeatureMI=np.array(FeatureMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(FeatureMI)
FeatureMI=scaled1#shape:7*7
#MeanMI,ConcatMI,FeatureMI

In [34]:
#3.5 Mean MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
#Index all of the rows and columns
DyMeanMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyMeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyMeanMI=np.array(DyMeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyMeanMI=DyMeanMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyMeanMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyMeanMI[i,:,:]=MEAN_imputer.fit_transform(DyMeanMI[i,:,:])
print(DyMeanMI)#Here DyMeanMI is done

(7266, 6, 10)


/content/DAMR/model.py:67: RuntimeWarning: invalid value encountered in divide
  c_normalized = c / float(np.sum(c))


KeyboardInterrupt: 

In [ ]:
#3.6 Concat MI-based dynamic adjacency matrix
#Index all of the rows and columns
DyConcatMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyConcatMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyConcatMI=np.array(DyConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyConcatMI=DyConcatMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyConcatMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyConcatMI[i,:,:]=MEAN_imputer.fit_transform(DyConcatMI[i,:,:])
print(DyConcatMI)#Here DyConcatMI is done


In [ ]:
#3.7 FFT-based sum batch's MI dynamic adjacency matrix
yf = abs(fft(COVID_features_aftslid[:,0,0]))
T=yf[:3]
T = [int(x) for x in T]
Term1=T[0]
Term2=T[1]
Term3=T[2]
#Step1: Calculate Adj in each batch
#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a.shape[0]):
    for i in range(Term1):
        data.append(a[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b.shape[0]):
    for i in range(Term2):
        data.append(b[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c.shape[0]):
    for i in range(Term3):
        data.append(c[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step3: Add all of the adjacency matrices
DyConcatFFTMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTMI.shape)#(473,7,7)

# Step4: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTMI[i,:,:] = scaler.fit_transform(DyConcatFFTMI[i,:,:])
print(DyConcatFFTMI)

In [35]:
print(COVID_features_aftslid.shape)

(7266, 6, 10)


[5370, 568, 109]


In [45]:
#3.8 FFT-based MI-based dynamic adjacency matrix
yf = abs(fft(COVID_features_aftslid[:,0,0]))
T=yf[:3]
T = [int(x) for x in T]
Term1=T[0]
Term2=T[1]
Term3=T[2]

#Step1: Calculate Adj in each window

#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2: Calculate MI's MI for periodic term 1
#Term1
temp=[]
for j in range(a.shape[0]):
    for i in range(a.shape[0]):
        temp.append(Global_MeanMI(a[j,:,:],a[i,:,:],2))#The first point MI's MI
temp=np.array(temp).reshape(a.shape[0],a.shape[0])#24*24

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(a.shape[0],a.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#24*24
print(index_all.shape)#24*24

all_term=5#Get the first 5 decomposition
a_all=np.zeros((a.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(a.shape[0]):
    for u in range(all_term):
        a_all[i,:,:]=a_all[i,:,:]+a[i,:,:]+index_all[i,u]*temp[i,u]
print(a_all.shape)#(24,7,7)

#Step2: Calculate MI's MI for periodic term 2
#Term2
temp=[]
for j in range(b.shape[0]):
    for i in range(b.shape[0]):
        temp.append(Global_MeanMI(b[j,:,:],b[i,:,:],2))#The first point MI's MI
temp=np.array(temp).reshape(b.shape[0],b.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(b.shape[0],b.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#12*12
print(index_all.shape)#12*12

all_term=5#Get the first 5 decomposition
b_all=np.zeros((b.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(b.shape[0]):
    for u in range(all_term):
        b_all[i,:,:]=b_all[i,:,:]+b[i,:,:]+index_all[i,u]*temp[i,u]
print(b_all.shape)#(12,7,7)

#Step2: Calculate MI's MI for periodic term 3
#Term3
temp=[]
for j in range(c.shape[0]):
    for i in range(c.shape[0]):
        temp.append(Global_MeanMI(c[j,:,:],c[i,:,:],2))#The first point MI's MI
temp=np.array(temp).reshape(c.shape[0],c.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(c.shape[0],c.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#5*5
print(index_all.shape)#5*5

all_term=5#Get the first 5 decomposition
c_all=np.zeros((c.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(c.shape[0]):
    for u in range(all_term):
        c_all[i,:,:]=c_all[i,:,:]+c[i,:,:]+index_all[i,u]*temp[i,u]
print(c_all.shape)#(5,7,7)

#Step4:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a_all.shape[0]):
    for i in range(Term1):
        data.append(a_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b_all.shape[0]):
    for i in range(Term2):
        data.append(b_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c_all.shape[0]):
    for i in range(Term3):
        data.append(c_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step5: Add all of the adjacency matrices
DyConcatFFTeachMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTeachMI.shape)#(473,7,7)

# Step6: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTeachMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTeachMI[i,:,:] = scaler.fit_transform(DyConcatFFTeachMI[i,:,:])
print(DyConcatFFTeachMI)

(2, 10, 10)
(10, 10, 13)
(13, 10, 10)


KeyboardInterrupt: 

In [ ]:
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =24
epochs = 100
seed = 42
verbose = 1

In [ ]:
## Define Model
inputs_ret = Input(shape=(COVID_X1_all_aftslid.shape[1], COVID_X1_all_aftslid.shape[2], ))#7*30=node*lags(Time to look back)
inputs_feat = Input(shape=(COVID_features_aftslid.shape[2], COVID_features_aftslid.shape[1], ))#7*6=Node*feature
inputs_adj = Input(shape=(Adj_dist.shape[1],Adj_dist.shape[1], ))#7*7=node*node
print(inputs_ret.shape)
print(inputs_feat.shape)
print(inputs_adj.shape)

In [ ]:
#GATinput:1.input nodes*lags 2.adj:nodes*nodes
#GAToutput:1.channels*att_heads2.Att_weights=nodes*heads*nodes
GAT_output_P, Att_weights_P =  GATConv(16, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=True,
                                   activation='relu', use_bias=False)([inputs_ret, inputs_adj])
Att_weights_P = tf.math.reduce_mean(Att_weights_P, axis=2)#reduce heads,it will be nodes*nodes
print(Att_weights_P)
GCN_output_P =  GCNConv(16, dropout_rate=0.3,
                        activation='relu', use_bias=True)([inputs_feat, inputs_adj])
diff_out = DiffusionConv(16, K=6, activation='tanh')([inputs_feat, Att_weights_P])#diff_out=nodes*channels
con_out_P_2 = Concatenate(axis=-1)([GAT_output_P, GCN_output_P,diff_out])
dense_P = Dense(8, activation='relu')(con_out_P_2)
dense_out = Dropout(0.2)(dense_P)
outputs = Dense(1)(dense_out)
model = Model(inputs=[inputs_ret, inputs_feat, inputs_adj], outputs=outputs)
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=["mae", "mape"])
model.summary()

In [ ]:
#Get 473*7*7 Adj_dist
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7
Adj_dist=pd.DataFrame(Adj_dist)
Adj_all = np.arange(len(COVID_Y1_all)*Adj_dist.shape[0]*Adj_dist.shape[0]).reshape(len(COVID_Y1_all),Adj_dist.shape[0],Adj_dist.shape[0])#(len,stations,stations)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_dist
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_dist)
data=np.array(data)
Adj_dist=data
#MeanMI
Adj_MeanMI=MeanMI
Adj_all = np.arange(len(COVID_Y1_all)*Adj_dist.shape[1]*Adj_dist.shape[1]).reshape(len(COVID_Y1_all),Adj_dist.shape[1],Adj_dist.shape[1])#(len,stations,stations)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_MeanMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_MeanMI)
data=np.array(data)
Adj_MeanMI=data
#ConcatMI
Adj_ConcatMI=ConcatMI
Adj_all = np.arange(len(COVID_Y1_all)*Adj_dist.shape[1]*Adj_dist.shape[1]).reshape(len(COVID_Y1_all),Adj_dist.shape[1],Adj_dist.shape[1])#(len,stations,stations)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_ConcatMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_ConcatMI)
data=np.array(data)
Adj_ConcatMI=data
#FeatureMI
Adj_FeatureMI=FeatureMI
Adj_all = np.arange(len(COVID_Y1_all)*Adj_dist.shape[1]*Adj_dist.shape[1]).reshape(len(COVID_Y1_all),Adj_dist.shape[1],Adj_dist.shape[1])#(len,stations,stations)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_FeatureMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_FeatureMI)
data=np.array(data)
Adj_FeatureMI=data


In [ ]:
print(COVID_features_aftslid.shape)#473*3*7
COVID_features_aftslid= np.transpose(COVID_features_aftslid,(0,2,1))#473*7*3
print(COVID_X1_all_aftslid.shape)#473*7*30
print(np.array(COVID_Y1_all).shape)#(473, 7, 1)
print(np.array(Adj_MeanMI).shape)#473*7*7
#Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled1 = scaler.fit_transform(COVID_Y1_all[:,:,0])
COVID_Y1_all[:,:,0]=scaled1

In [ ]:
#3.9 ->MeanMI approach(3.1+3.2+3.5+3.8)
coe1=0.8
coe2=0.1
coe3=0.1
coe4=0.1

###innovation: (random search)

Adj_all_MeanMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# scaler = MinMaxScaler(feature_range=(-1, 1))
# for i in range(Adj_all_MeanMI.shape[0]):
#     Adj_all_MeanMI[i,:,:]=scaler.fit_transform(Adj_all_MeanMI[i,:,:])
#3.10 ->()ConcatMI approach
Adj_all_ConcatMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# for i in range(Adj_all_ConcatMI.shape[0]):
#     Adj_all_ConcatMI[i,:,:]=scaler.fit_transform(Adj_all_ConcatMI[i,:,:])

In [ ]:
DyConcatMI=Adj_all_ConcatMI
x_train=COVID_X1_all_aftslid[:int(len(COVID_X1_all_aftslid)*0.8),:,:]
x_val=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.8):int(len(COVID_X1_all_aftslid)*0.9),:,:]
x_test=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.9):,:,:]
features_train=COVID_features_aftslid[:int(len(COVID_features_aftslid)*0.8),:,:]
features_val=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.8):int(len(COVID_features_aftslid)*0.9),:,:]
features_test=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.9):,:,:]
adj_train=DyConcatMI[:int(len(DyConcatMI)*0.8),:,:]
adj_val=DyConcatMI[int(len(DyConcatMI)*0.8):int(len(DyConcatMI)*0.9),:,:]
adj_test=DyConcatMI[int(len(DyConcatMI)*0.9):,:,:]
y_train=COVID_Y1_all[:int(len(COVID_Y1_all)*0.8),:,:]
y_val=COVID_Y1_all[int(len(COVID_Y1_all)*0.8):int(len(COVID_Y1_all)*0.9),:,:]
y_test=COVID_Y1_all[int(len(COVID_Y1_all)*0.9):,:,:]

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero


# hists = model.fit(x= [x_train, features_train, adj_train],
#                   y= y_train, verbose=verbose, epochs=500,
#                   batch_size=batch_size, #validation_split=0.1,
#                   validation_data=([x_val, features_val, adj_val], y_val),
#                   #test_data=([x_test, features_test, adj_test], y_test),
#   )

##If you want to train faster, try earlystopping.

hists = model.fit(x= [x_train, features_train, adj_train],
                  y= y_train, verbose=verbose, epochs=5000,
                  batch_size=batch_size, #validation_split=0.1,
                  validation_data=([x_val, features_val, adj_val], y_val),
                  #test_data=([x_test, features_test, adj_test], y_test),
                    callbacks=[EarlyStopping(#monitor="val_mean_absolute_error",
                     monitor="val_loss", patience=20, restore_best_weights=True)],
  )

In [ ]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return np.mean(diff, axis=-1).mean()
###Write the function of mre
def mre(y_true, y_pred):
    return np.sum(np.abs(y_pred - y_true))/np.sum(np.abs(y_true))
def get_metrics(y, yp):
    return {

        "mae": np.round(mae(y, yp), 4),
        "mape": np.round(mape(y, yp),4),
        "mre": np.round(mre(y, yp),4),
        "rmse": np.round(rmse(y, yp), 4),
        #"MPE": np.round(MPE(y, yp), 4),
        #"R2": np.round(R_squared(y, yp),4)
    }

# scaled1 = scaler.fit_transform(MeanMI)
#Get the tensor of test set after masking it.
test_predicted = model.predict([x_test, features_test, adj_test])
get_metrics(y_test.flatten(), test_predicted.flatten())